# Further Exploration into Lifetime Return Differences

With data limited to 1 year, previous calculations on lifetime return of referred vs nonreferred accounts may prove inaccurate due to lack of data of newly created accounts. This section looks into lifetime return of accounts that were activated earlier in the provided data. 

In [1]:
import pandas as pd
import numpy as np
data1 = "1_paid_marketing.csv"
data2 = "2_hcp_data.csv"
data3 = "3_shifts_data.csv"
campaigns = pd.read_csv(data1)
hcp = pd.read_csv(data2)
shifts = pd.read_csv(data3)

In [2]:
# Convert datatype of 'lifetime shifts' and 'first shift time'
hcp.LIFETIME_SHIFTS = pd.to_numeric(hcp['LIFETIME_SHIFTS'], errors='coerce')
hcp.FIRST_SHIFT_TIME = pd.to_datetime(hcp['FIRST_SHIFT_TIME'], errors='coerce')

In [3]:
# Split up accounts based on time of first shift. Either 1st or 2nd half of 2022
hcp = hcp.set_index('FIRST_SHIFT_TIME')
hcp1 = hcp.loc['2022-01':'2022-06'].copy()
hcp2 = hcp.loc['2022-07':'2023-01'].copy()

In [4]:
# Showing how accounts made active in 1st half worked more shifts
print('Avg shifts of accounts activated in 1st half: ' + str(round(hcp1['LIFETIME_SHIFTS'].mean(), 2)))
print('Avg shifts of accounts activated in 2nd half: ' + str(round(hcp2['LIFETIME_SHIFTS'].mean(), 2)))

Avg shifts of accounts activated in 1st half: 20.3
Avg shifts of accounts activated in 2nd half: 14.66


In [5]:
tot_accounts = hcp1['HCP_ID'].count()
unique_referrers = hcp1['REFERRER'].value_counts().count()


print('In the first half of 2022:')
print('Total accounts created: ' + str(tot_accounts))
print('Total unique referrers: ' + str(unique_referrers))


In the first half of 2022:
Total accounts created: 6089
Total unique referrers: 1536


In [6]:
hcp1['REFERRER'].fillna('None', inplace=True)
hcp1.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY
FIRST_SHIFT_TIME,,,,,,,,,,
2022-06-03,61e75e34dfb86b018a0d7cf7,"Minneapolis-St. Paul-Bloomington, MN-WI",CNA,1/19/22,5/24/22,5/24/22,6/1/22,49.0,None,15595984274
2022-06-29,62b2499e52b98c01b901cd9d,"Chicago-Naperville-Elgin, IL-IN-WI",RN,6/21/22,6/22/22,6/22/22,6/29/22,2.0,62a48ae5691b0409a3c93073,16802693037
2022-02-28,61f962a1d6a2b2018a8e7159,"Riverside-San Bernardino-Ontario, CA",CNA,2/1/22,2/1/22,2/16/22,2/17/22,70.0,None,18530777721
2022-05-20,626fd8342ccf7701c2d9d11d,"Cleveland-Elyria, OH",CNA,5/2/22,5/2/22,5/13/22,5/15/22,23.0,None,17631861264
2022-06-20,62499dd3db204601acbd8311,"Deltona-Daytona Beach-Ormond Beach, FL",CNA,4/3/22,4/3/22,4/3/22,4/3/22,2.0,623a97fbca145401acdf587c,18581898213


In [7]:
# Find only active accounts made without a referral
nonref_active = hcp1.loc[hcp1['REFERRER'] == 'None'].copy()
nonref_active = nonref_active.reset_index(drop=True)
nonref_active.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY
0,61e75e34dfb86b018a0d7cf7,"Minneapolis-St. Paul-Bloomington, MN-WI",CNA,1/19/22,5/24/22,5/24/22,6/1/22,49.0,None,15595984274
1,61f962a1d6a2b2018a8e7159,"Riverside-San Bernardino-Ontario, CA",CNA,2/1/22,2/1/22,2/16/22,2/17/22,70.0,None,18530777721
2,626fd8342ccf7701c2d9d11d,"Cleveland-Elyria, OH",CNA,5/2/22,5/2/22,5/13/22,5/15/22,23.0,None,17631861264
3,61fa31443e9933018b2ac502,"Bucyrus-Galion, OH",CNA,2/2/22,5/11/22,5/12/22,5/12/22,37.0,None,15595984274
4,627f2dbe3ce6ce01bde9bd20,"Phoenix-Mesa-Chandler, AZ",RN,5/14/22,5/14/22,5/18/22,5/18/22,15.0,None,18530777721


In [8]:
# Calculate avg lifetime value of non-referral active accounts
nraa_profit = 0
nraa_hcp = nonref_active['HCP_ID'].count()

cna_rate = 5
lvn_rate = 10
rn_rate = 15

for x in np.arange(nraa_hcp):
    if nonref_active.iloc[x,2] == 'CNA':
        nraa_profit += nonref_active.iloc[x,7] * cna_rate * 8
    elif nonref_active.iloc[x,2] == 'LVN':
        nraa_profit += nonref_active.iloc[x,7] * lvn_rate * 8
    elif nonref_active.iloc[x,2] == 'RN':
        nraa_profit += nonref_active.iloc[x,7] * rn_rate * 8

avg_nr_profit = round(nraa_profit / nraa_hcp, 2)
avg_nr_shifts = round(nonref_active['LIFETIME_SHIFTS'].mean(), 2)

print('Average profits from nonreferred workers: ' + str(avg_nr_profit))
print('Average shifts for nonreferred workers: ' + str(avg_nr_shifts))

Average profits from nonreferred workers: 1219.06
Average shifts for nonreferred workers: 20.17


In [9]:
# Find only active accounts made WITH a referral
ref_active = hcp1.loc[hcp1['REFERRER'] != 'None'].copy()
ref_active = ref_active.reset_index(drop=True)
ref_active.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY
0,62b2499e52b98c01b901cd9d,"Chicago-Naperville-Elgin, IL-IN-WI",RN,6/21/22,6/22/22,6/22/22,6/29/22,2.0,62a48ae5691b0409a3c93073,16802693037
1,62499dd3db204601acbd8311,"Deltona-Daytona Beach-Ormond Beach, FL",CNA,4/3/22,4/3/22,4/3/22,4/3/22,2.0,623a97fbca145401acdf587c,18581898213
2,62695ce9a0811501b63d432a,"Youngstown-Warren-Boardman, OH-PA",CNA,4/27/22,4/27/22,4/27/22,4/29/22,22.0,618348c536650a0186fc6c50,18672087778
3,627bf7aeefb13c01c188a565,"New York-Newark-Jersey City, NY-NJ-PA",CNA,5/11/22,5/11/22,5/11/22,5/14/22,2.0,61fd74fae8b6ff01af8d85d7,na
4,62476ac9ca832e01aa8d1246,"Napa, CA",CNA,4/1/22,4/4/22,4/22/22,4/29/22,58.0,616453b0ac801c0169f80702,na


In [10]:
# Calculate avg lifetime value of referral active accounts
raa_profit = 0
raa_hcp = ref_active['HCP_ID'].count()

for x in np.arange(raa_hcp):
    if ref_active.iloc[x,2] == 'CNA':
        raa_profit += ref_active.iloc[x,7] * cna_rate * 8
    elif ref_active.iloc[x,2] == 'LVN':
        raa_profit += ref_active.iloc[x,7] * lvn_rate * 8
    elif ref_active.iloc[x,2] == 'RN':
        raa_profit += ref_active.iloc[x,7] * rn_rate * 8

avg_ref_profit = round(raa_profit / raa_hcp, 2)
avg_ref_shifts = round(ref_active['LIFETIME_SHIFTS'].mean(), 2)

print('Average profits from referred workers: ' + str(avg_ref_profit))
print('Average shifts for referred workers: ' + str(avg_ref_shifts))

Average profits from referred workers: 1141.08
Average shifts for referred workers: 20.57


# Set up data for LA and Nashville analysis

In [11]:
# Find unique values for MSA
hcp1['MSA'].value_counts()

Chicago-Naperville-Elgin, IL-IN-WI             1008
Los Angeles-Long Beach-Anaheim, CA              454
Cleveland-Elyria, OH                            335
Riverside-San Bernardino-Ontario, CA            262
New York-Newark-Jersey City, NY-NJ-PA           224
Dallas-Fort Worth-Arlington, TX                 173
San Diego-Chula Vista-Carlsbad, CA              157
Philadelphia-Camden-Wilmington, PA-NJ-DE-MD     142
Atlanta-Sandy Springs-Alpharetta, GA            133
Sacramento-Roseville-Folsom, CA                 131
Phoenix-Mesa-Chandler, AZ                       126
Detroit-Warren-Dearborn, MI                     124
Las Vegas-Henderson-Paradise, NV                120
San Francisco-Oakland-Berkeley, CA              112
Milwaukee-Waukesha, WI                          110
Houston-The Woodlands-Sugar Land, TX            104
Pittsburgh, PA                                  102
Columbus, OH                                     95
Tampa-St. Petersburg-Clearwater, FL              91
Orlando-Kiss

In [12]:
# Splitting MSA into state for easier filtering below
hcp1[['MSA', 'state']] = hcp1['MSA'].str.split(', ', 1, expand=True)
hcp1

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY,state
FIRST_SHIFT_TIME,,,,,,,,,,,
2022-06-03,61e75e34dfb86b018a0d7cf7,Minneapolis-St. Paul-Bloomington,CNA,1/19/22,5/24/22,5/24/22,6/1/22,49.0,None,15595984274,MN-WI
2022-06-29,62b2499e52b98c01b901cd9d,Chicago-Naperville-Elgin,RN,6/21/22,6/22/22,6/22/22,6/29/22,2.0,62a48ae5691b0409a3c93073,16802693037,IL-IN-WI
2022-02-28,61f962a1d6a2b2018a8e7159,Riverside-San Bernardino-Ontario,CNA,2/1/22,2/1/22,2/16/22,2/17/22,70.0,None,18530777721,CA
2022-05-20,626fd8342ccf7701c2d9d11d,Cleveland-Elyria,CNA,5/2/22,5/2/22,5/13/22,5/15/22,23.0,None,17631861264,OH
2022-06-20,62499dd3db204601acbd8311,Deltona-Daytona Beach-Ormond Beach,CNA,4/3/22,4/3/22,4/3/22,4/3/22,2.0,623a97fbca145401acdf587c,18581898213,FL
2022-05-13,61fa31443e9933018b2ac502,Bucyrus-Galion,CNA,2/2/22,5/11/22,5/12/22,5/12/22,37.0,None,15595984274,OH
2022-05-02,62695ce9a0811501b63d432a,Youngstown-Warren-Boardman,CNA,4/27/22,4/27/22,4/27/22,4/29/22,22.0,618348c536650a0186fc6c50,18672087778,OH-PA
2022-05-19,627f2dbe3ce6ce01bde9bd20,Phoenix-Mesa-Chandler,RN,5/14/22,5/14/22,5/18/22,5/18/22,15.0,None,18530777721,AZ
2022-03-30,621f2a1678c27801ac191b9b,Los Angeles-Long Beach-Anaheim,CNA,3/2/22,3/2/22,3/11/22,3/24/22,21.0,None,na,CA


In [13]:
# Replace empty referrers with 'None' for easier filtering later
hcp1['REFERRER'].fillna('None', inplace=True)
hcp1.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY,state
FIRST_SHIFT_TIME,,,,,,,,,,,
2022-06-03,61e75e34dfb86b018a0d7cf7,Minneapolis-St. Paul-Bloomington,CNA,1/19/22,5/24/22,5/24/22,6/1/22,49.0,None,15595984274,MN-WI
2022-06-29,62b2499e52b98c01b901cd9d,Chicago-Naperville-Elgin,RN,6/21/22,6/22/22,6/22/22,6/29/22,2.0,62a48ae5691b0409a3c93073,16802693037,IL-IN-WI
2022-02-28,61f962a1d6a2b2018a8e7159,Riverside-San Bernardino-Ontario,CNA,2/1/22,2/1/22,2/16/22,2/17/22,70.0,None,18530777721,CA
2022-05-20,626fd8342ccf7701c2d9d11d,Cleveland-Elyria,CNA,5/2/22,5/2/22,5/13/22,5/15/22,23.0,None,17631861264,OH
2022-06-20,62499dd3db204601acbd8311,Deltona-Daytona Beach-Ormond Beach,CNA,4/3/22,4/3/22,4/3/22,4/3/22,2.0,623a97fbca145401acdf587c,18581898213,FL


In [14]:
# Creating separate dataframes for California and Tennessee
cal_hcp = hcp1.loc[hcp1['state']=='CA']
ten_hcp = hcp1.loc[hcp1['state']=='TN']

In [15]:
# Easier to see unique values for Cal, and Ten below
cal_hcp['MSA'].value_counts()

Los Angeles-Long Beach-Anaheim      454
Riverside-San Bernardino-Ontario    262
San Diego-Chula Vista-Carlsbad      157
Sacramento-Roseville-Folsom         131
San Francisco-Oakland-Berkeley      112
Stockton                             68
Modesto                              45
Fresno                               44
San Jose-Sunnyvale-Santa Clara       32
Vallejo                              19
Merced                               10
Yuba City                             7
Salinas                               6
Bakersfield                           6
Oxnard-Thousand Oaks-Ventura          5
Napa                                  5
Santa Rosa-Petaluma                   4
Hanford-Corcoran                      3
Visalia                               3
Chico                                 2
Madera                                2
Santa Cruz-Watsonville                1
Clearlake                             1
Red Bluff                             1
Sonora                                1


In [16]:
ten_hcp['MSA'].value_counts()

Nashville-Davidson--Murfreesboro--Franklin    36
Knoxville                                      6
Jackson                                        4
Brownsville                                    3
Tullahoma-Manchester                           2
Cleveland                                      1
Morristown                                     1
Greeneville                                    1
Name: MSA, dtype: int64

In [17]:
# Create LA and Nash dataframes
la_hcp = cal_hcp.loc[cal_hcp['MSA']=='Los Angeles-Long Beach-Anaheim'].copy()
nash_hcp = ten_hcp.loc[ten_hcp['MSA']=='Nashville-Davidson--Murfreesboro--Franklin'].copy()

In [18]:
# Store active accounts in LA and Nash
la_working_accounts = la_hcp['HCP_ID'].count()
nash_working_accounts = nash_hcp['HCP_ID'].count()

print('Active accounts in LA: ' + str(la_working_accounts))
print('Active accounts in Nashville: ' + str(nash_working_accounts))

Active accounts in LA: 454
Active accounts in Nashville: 36


# Calculating return for referral vs non in LA

In [19]:
# Find active accounts made without a referral
la_nonref_active = la_hcp.loc[la_hcp['REFERRER']=='None'].copy()
la_nonref_active = la_nonref_active.reset_index(drop=True)
la_nonref_active.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY,state
0,621f2a1678c27801ac191b9b,Los Angeles-Long Beach-Anaheim,CNA,3/2/22,3/2/22,3/11/22,3/24/22,21.0,None,na,CA
1,626ad36d7e73ce01c14cb6e8,Los Angeles-Long Beach-Anaheim,CNA,4/28/22,4/28/22,NaN,4/28/22,20.0,None,na,CA
2,61ea2b454c00ba018aee6f9f,Los Angeles-Long Beach-Anaheim,LVN,1/21/22,1/21/22,2/5/22,6/5/22,2.0,None,na,CA
3,62674aad24773701af443671,Los Angeles-Long Beach-Anaheim,CNA,4/26/22,4/26/22,4/27/22,4/28/22,27.0,None,na,CA
4,6259b09968ff6601aec357c1,Los Angeles-Long Beach-Anaheim,CNA,4/15/22,4/15/22,12/30/22,5/13/22,15.0,None,na,CA


In [20]:
# Calculate avg lifetime value of non-referral active accounts
la_nraa_profit = 0
la_nraa_hcp = la_nonref_active['HCP_ID'].count()

cna_rate = 5
lvn_rate = 10
rn_rate = 15

for x in np.arange(la_nraa_hcp):
    if la_nonref_active.iloc[x,2] == 'CNA':
        la_nraa_profit += la_nonref_active.iloc[x,7] * cna_rate * 8
    elif la_nonref_active.iloc[x,2] == 'LVN':
        la_nraa_profit += la_nonref_active.iloc[x,7] * lvn_rate * 8
    elif la_nonref_active.iloc[x,2] == 'RN':
        la_nraa_profit += la_nonref_active.iloc[x,7] * rn_rate * 8

la_avg_nr_profit = round(la_nraa_profit / la_nraa_hcp, 2)
la_avg_nr_shifts = round(la_nonref_active['LIFETIME_SHIFTS'].mean(), 2)

print('Average profits from nonreferred workers in LA: ' + str(la_avg_nr_profit))
print('Average shifts for nonreferred workers in LA: ' + str(la_avg_nr_shifts))

Average profits from nonreferred workers in LA: 1218.28
Average shifts for nonreferred workers in LA: 21.01


In [21]:
# Find active accounts made WITH a referral
la_ref_active = la_hcp.loc[la_hcp['REFERRER'] != 'None'].copy()
la_ref_active = la_ref_active.reset_index(drop=True)
la_ref_active.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY,state
0,62b23d258e735a01bc79773e,Los Angeles-Long Beach-Anaheim,LVN,6/21/22,6/21/22,6/22/22,6/22/22,10.0,6286843c9a73a501c128749c,na,CA
1,626cbb5947d5a701c2b5e9a0,Los Angeles-Long Beach-Anaheim,CNA,4/30/22,4/30/22,5/1/22,5/7/22,2.0,6190591a677f8e01862f55d9,15526159364,CA
2,620fc496da7cf401a9cfea23,Los Angeles-Long Beach-Anaheim,LVN,2/18/22,2/18/22,8/10/22,6/1/22,22.0,60fd1e068b75e3016625a5a1,na,CA
3,625d97638fd22d01ac945dde,Los Angeles-Long Beach-Anaheim,RN,4/18/22,4/18/22,4/18/22,4/22/22,2.0,61bec61fd20fe401864fdb1e,13743509691,CA
4,61f3b52e850e71018ad9ce0f,Los Angeles-Long Beach-Anaheim,CNA,1/28/22,1/28/22,1/28/22,1/28/22,39.0,61cb1122e9cdd0018a2a7df3,15645881423,CA


In [22]:
la_raa_profit = 0
la_raa_hcp = la_ref_active['HCP_ID'].count()

cna_rate = 5
lvn_rate = 10
rn_rate = 15

for x in np.arange(la_raa_hcp):
    if la_ref_active.iloc[x,2] == 'CNA':
        la_raa_profit += la_ref_active.iloc[x,7] * cna_rate * 8
    elif la_ref_active.iloc[x,2] == 'LVN':
        la_raa_profit += la_ref_active.iloc[x,7] * lvn_rate * 8
    elif la_ref_active.iloc[x,2] == 'RN':
        la_raa_profit += la_ref_active.iloc[x,7] * rn_rate * 8

la_avg_ref_profit = round(la_raa_profit / la_raa_hcp, 2)
la_avg_ref_shifts = round(la_ref_active['LIFETIME_SHIFTS'].mean(), 2)

print('Average profits from referred workers in LA: ' + str(la_avg_ref_profit))
print('Average shifts for referred workers in LA: ' + str(la_avg_ref_shifts))

Average profits from referred workers in LA: 1065.62
Average shifts for referred workers in LA: 20.06


# Calculating return for referral vs non in Nash

In [23]:
# Find active accounts made without a referral
nash_nonref_active = nash_hcp.loc[nash_hcp['REFERRER']=='None'].copy()
nash_nonref_active = nash_nonref_active.reset_index(drop=True)
nash_nonref_active.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY,state
0,61de4d41001d43018b981d89,Nashville-Davidson--Murfreesboro--Franklin,CNA,1/12/22,1/12/22,3/25/22,3/30/22,10.0,None,na,TN
1,61f7033bfac74e018ae365ce,Nashville-Davidson--Murfreesboro--Franklin,LVN,1/30/22,1/30/22,1/31/22,3/11/22,20.0,None,na,TN
2,629a155292a26601c97334d9,Nashville-Davidson--Murfreesboro--Franklin,LVN,6/3/22,6/3/22,11/17/22,6/13/22,20.0,None,na,TN
3,629e984fa6a3ae01c224be7d,Nashville-Davidson--Murfreesboro--Franklin,LVN,6/7/22,6/7/22,6/7/22,6/7/22,51.0,None,424720106,TN
4,62605fc2ccf4b101ad934408,Nashville-Davidson--Murfreesboro--Franklin,RN,4/20/22,4/20/22,4/20/22,4/21/22,6.0,None,na,TN


In [24]:
# Calculate avg lifetime value of non-referral active accounts
nash_nraa_profit = 0
nash_nraa_hcp = nash_nonref_active['HCP_ID'].count()

cna_rate = 5
lvn_rate = 10
rn_rate = 15

for x in np.arange(nash_nraa_hcp):
    if nash_nonref_active.iloc[x,2] == 'CNA':
        nash_nraa_profit += nash_nonref_active.iloc[x,7] * cna_rate * 8
    elif nash_nonref_active.iloc[x,2] == 'LVN':
        nash_nraa_profit += nash_nonref_active.iloc[x,7] * lvn_rate * 8
    elif nash_nonref_active.iloc[x,2] == 'RN':
        nash_nraa_profit += nash_nonref_active.iloc[x,7] * rn_rate * 8

nash_avg_nr_profit = round(nash_nraa_profit / nash_nraa_hcp, 2)
nash_avg_nr_shifts = round(nash_nonref_active['LIFETIME_SHIFTS'].mean(), 2)

print('Average profits from nonreferred workers in Nashville: ' + str(nash_avg_nr_profit))
print('Average shifts for nonreferred workers in Nashville: ' + str(nash_avg_nr_shifts))

Average profits from nonreferred workers in Nashville: 1124.44
Average shifts for nonreferred workers in Nashville: 17.39


In [25]:
# Find active accounts made WITH a referral
nash_ref_active = nash_hcp.loc[nash_hcp['REFERRER']!='None'].copy()
nash_ref_active = nash_ref_active.reset_index(drop=True)
nash_ref_active.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY,state
0,626b813debba6601c3de12f5,Nashville-Davidson--Murfreesboro--Franklin,CNA,4/29/22,5/2/22,6/1/22,6/3/22,21.0,624f7ec79cf773019fcd1adf,na,TN
1,629d0b908bc8f701c29200f2,Nashville-Davidson--Murfreesboro--Franklin,CNA,6/5/22,6/6/22,6/7/22,6/8/22,22.0,626df239d9a36e01c7e5589a,16920183767,TN
2,622a3f7541ba9a01a29baeb7,Nashville-Davidson--Murfreesboro--Franklin,CNA,3/10/22,3/26/22,3/28/22,4/6/22,42.0,62389a98a1fad101ab2cc878,na,TN
3,624f3e2f9cf773019fbf90cc,Nashville-Davidson--Murfreesboro--Franklin,LVN,4/7/22,5/1/22,5/2/22,5/2/22,21.0,624f7ec79cf773019fcd1adf,na,TN
4,61f673d0360033018a084a45,Nashville-Davidson--Murfreesboro--Franklin,CNA,1/30/22,1/30/22,3/9/22,3/18/22,8.0,6135d0dbbb5326016685b0c2,na,TN


In [26]:
nash_raa_profit = 0
nash_raa_hcp = nash_ref_active['HCP_ID'].count()

cna_rate = 5
lvn_rate = 10
rn_rate = 15

for x in np.arange(nash_raa_hcp):
    if nash_ref_active.iloc[x,2] == 'CNA':
        nash_raa_profit += nash_ref_active.iloc[x,7] * cna_rate * 8
    elif nash_ref_active.iloc[x,2] == 'LVN':
        nash_raa_profit += nash_ref_active.iloc[x,7] * lvn_rate * 8
    elif nash_ref_active.iloc[x,2] == 'RN':
        nash_raa_profit += nash_ref_active.iloc[x,7] * rn_rate * 8

nash_avg_ref_profit = round(nash_raa_profit / nash_raa_hcp, 2)
nash_avg_ref_shifts = round(nash_ref_active['LIFETIME_SHIFTS'].mean(), 2)

print('Average profits from referred workers in Nashville: ' + str(nash_avg_ref_profit))
print('Average shifts for referred workers in Nashville: ' + str(nash_avg_ref_shifts))

Average profits from referred workers in Nashville: 988.89
Average shifts for referred workers in Nashville: 21.89


# Other Referral Program Statistics

In [27]:
# Remove accounts without referrals
la_hcp = la_hcp.loc[la_hcp['REFERRER'] != 'None']
nash_hcp = nash_hcp.loc[nash_hcp['REFERRER'] != 'None']

In [28]:
la_conv_refers = la_hcp['REFERRER'].count()
nash_conv_refers = nash_hcp['REFERRER'].count()

print('Active accounts with a referral in LA: ' + str(la_conv_refers))
print('Active accounts with a referral in Nashville: ' + str(nash_conv_refers))

Active accounts with a referral in LA: 128
Active accounts with a referral in Nashville: 18


In [29]:
la_ref_workers = round(la_conv_refers / la_working_accounts, 2)
nash_ref_workers = round(nash_conv_refers / nash_working_accounts, 2)

print('Percent of active accounts with a referral in LA: ' + str(la_ref_workers))
print('Percent of active accounts with a referral in Nashville: ' + str(nash_ref_workers))

Percent of active accounts with a referral in LA: 0.28
Percent of active accounts with a referral in Nashville: 0.5


In [30]:
print('Avg shifts of accounts activated in 1st half: ' + str(round(hcp1['LIFETIME_SHIFTS'].mean(), 2)))
print('Avg shifts of accounts activated in 2nd half: ' + str(round(hcp2['LIFETIME_SHIFTS'].mean(), 2)))
print('')
print('')
print('In the first half of 2022:')
print('Total accounts created: ' + str(tot_accounts))
print('Total unique referrers: ' + str(unique_referrers))
print('')
print('Active accounts in LA: ' + str(la_working_accounts))
print('Active accounts in Nashville: ' + str(nash_working_accounts))
print('')
print('Average profits from nonreferred workers: ' + str(avg_nr_profit))
print('Average shifts for nonreferred workers: ' + str(avg_nr_shifts))
print('')
print('Average profits from nonreferred workers in LA: ' + str(la_avg_nr_profit))
print('Average shifts for nonreferred workers in LA: ' + str(la_avg_nr_shifts))
print('')
print('Average profits from nonreferred workers in Nashville: ' + str(nash_avg_nr_profit))
print('Average shifts for nonreferred workers in Nashville: ' + str(nash_avg_nr_shifts))
print('')
print('Average profits from referred workers: ' + str(avg_ref_profit))
print('Average shifts for referred workers: ' + str(avg_ref_shifts))
print('')
print('Average profits from referred workers in LA: ' + str(la_avg_ref_profit))
print('Average shifts for referred workers in LA: ' + str(la_avg_ref_shifts))
print('')
print('Average profits from referred workers in Nashville: ' + str(nash_avg_ref_profit))
print('Average shifts for referred workers in Nashville: ' + str(nash_avg_ref_shifts))

Avg shifts of accounts activated in 1st half: 20.3
Avg shifts of accounts activated in 2nd half: 14.66


In the first half of 2022:
Total accounts created: 6089
Total unique referrers: 1536

Active accounts in LA: 454
Active accounts in Nashville: 36

Average profits from nonreferred workers: 1219.06
Average shifts for nonreferred workers: 20.17

Average profits from nonreferred workers in LA: 1218.28
Average shifts for nonreferred workers in LA: 21.01

Average profits from nonreferred workers in Nashville: 1124.44
Average shifts for nonreferred workers in Nashville: 17.39

Average profits from referred workers: 1141.08
Average shifts for referred workers: 20.57

Average profits from referred workers in LA: 1065.62
Average shifts for referred workers in LA: 20.06

Average profits from referred workers in Nashville: 988.89
Average shifts for referred workers in Nashville: 21.89
